In [39]:
import os, sys
sys.path.append('/home/obed/dev/CVLoc')
from connectivity import Edge, Digraph
import pandas as pd 
import folium

In [40]:
def BFS2(graph, start, depth):
    """Assumes graph is a Digraph; start and end are nodes
    Returns a shortest path from start to end in graph"""
    initPath = [start]
    pathQueue = [initPath]
    nodeSet = [start]

    while len(pathQueue) != 0:
        #Get and remove oldest element in pathQueue
        tmpPath = pathQueue.pop(0)
        lastNode = tmpPath[-1]
        n = len(tmpPath)
        print(n, end="\r")
        if n == depth:
            return nodeSet
        for nextNode in graph.childrenOf(lastNode):
            
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
            
            if nextNode not in nodeSet:
                nodeSet = nodeSet + [nextNode]

    return None

In [41]:
def saveSet(nodeList, dirPath, city):
    columns = ["index","pano_id","yaw","lat","lon"]
    frame = pd.DataFrame(columns = columns)
    n = len(nodeList)
    for i, idx in enumerate(nodeList):
        data = {'index': idx, 
                'pano_id': nodesFrame.loc[idx,'pano_id'], 
                'yaw':nodesFrame.loc[idx,'yaw'],
                'lat': nodesFrame.loc[idx,'lat'],
                'lon': nodesFrame.loc[idx,'lon']}
        
        frame = frame.append(data, ignore_index=True) 
        outFile = os.path.join(dirPath, 'BSFset.csv')
        frame.to_csv(outFile, header=False, index=False)
        print('saved {}/{}'.format(i, n), end="\r")
    return frame

In [42]:
def printPath(path):
    """Assumes path is a list of nodes"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result

# Read metadata frames

In [43]:
dirPath = os.path.join( os.environ['datasets'], 'streetlearn')
city = 'manhattan'

# nodes frame
nodesFile = os.path.join(dirPath, 'jpegs_' + city +'_2019', 'nodes.txt')
names = ["pano_id", "yaw", "lat", "lon"]
nodesFrame = pd.read_csv(nodesFile, names=names)

# links frame
linksFile = os.path.join(dirPath, 'jpegs_' + city +'_2019', 'links.txt')
names = ["src", "bearing", "dst"]
linksFrame = pd.read_csv(linksFile, names=names)


# Add nodes to the graph

In [44]:
# Create a dictionary from pano id to index
index = {}

# Add nodes to graph
graph = Digraph()

for i in range(len(nodesFrame)):
    pano_id = nodesFrame.loc[i, "pano_id"]
    index[pano_id] = i 
    graph.addNode(i)
    print(i, end = "\r")

In [45]:
# Add links to graph

print("This may take a while ... time for a cup of tea!")
for i in range(len(linksFrame)):
    linkRow = linksFrame.loc[i, :]
    src = index[linkRow['src']]
    dst = index[linkRow['dst']]
    edge = Edge(src, dst)
    graph.addEdge(edge)
    print(i, end = "\r")

This may take a while ... time for a cup of tea!


# Perform BFS

In [46]:
start = '6rIMyvAZUW4sT3ffqYOg0w'
start_index = index[start]
nodeList = BFS2(graph, start_index, 120)

# Save the resulting points in a csv file and show the set on the map

In [35]:
print("Points in the set", len(nodeList))
frame = saveSet(nodeList, dirPath, city) 

Points in the set 3364


In [36]:
# Put a marker at the starting pano_id

startPoint = [frame.loc[0, 'lat'], frame.loc[0,'lon']]
map = folium.Map(location=startPoint, zoom_start=17)
tooltip = 'Click me!'
folium.Marker(startPoint, popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(map)

In [37]:
# Add rest of markers

tooltip = 'Click me!'
for i in range(len(frame)):
    point = [frame.loc[i,'lat'], frame.loc[i,'lon']]
    folium.CircleMarker(
        location=point,
        radius=5,
        #popup='Laurelhurst Park',
        color='#3186cc',
        fill=False,
        fill_color='#3186cc'
    ).add_to(map)

In [38]:
# Show the map
map